MAKE A BACKUP / COPY OF THIS WHENEVER UPDATING. 

This colab file is shared with multiple people, so anything you do on here may be overwritten (colab does not work in the same way as google docs, and it is easy to overwrite someone else's save). Creating a backup each time you make an important update will prevent your work from being deleted. Working on this file is fine. To prevent overwriting, make sure to use your own cell

This will contain the newest version of underlords AI. Info can be posted here on what to work on. The old code (which we will need to revamp) is on the github, which can be found on the discord.

# Simulator Class 

The following JSON file contains info about the Underlords pieces which will be used in making the simulator

https://github.com/SteamDatabase/GameTracking-Underlords/blob/master/game/dac/pak01_dir/scripts/units.json

Load the contents of the JSON 

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import json

file_path = 'units.json'
url = 'https://raw.githubusercontent.com/SteamDatabase/GameTracking-Underlords/master/game/dac/pak01_dir/scripts/units.json'

In [0]:
# Downloads a json frile from an URL and save to file_path
def downloads_json(url, file_path):
  import requests
  myfile = requests.get(url, allow_redirects=True)
  open(file_path, 'wb').write(myfile.content)

# Load json data to a python dictionary from file_path
def load_json(file_path):  
  import json 
  return json.load(open(file_path))
  

# Fetch data: download then load a json to a dictionary
def fetch_data(url, file_path='sample.json'):
  downloads_json(url, file_path)
  return load_json(file_path)


In [0]:
unit_info = fetch_data(url, file_path)

In [6]:
print(unit_info)

{'abaddon': {'abilities': ['abaddon_aphotic_shield'], 'armor': 10, 'attackAnimationPoint': 0.56, 'attackRange': 1, 'attackRate': 1.5, 'damageMin': [45, 90, 216], 'damageMax': [55, 110, 264], 'displayName': '#dac_hero_name_abaddon', 'dota_unit_name': 'npc_dota_hero_abaddon', 'draftTier': 3, 'goldCost': 3, 'health': [1500, 3000, 6000], 'id': 32, 'hintWords': ['#dac_hero_hint_melee', '#dac_hero_hint_explosive_shield'], 'keywords': 'undead knight', 'magicResist': 0, 'maxmana': 100, 'model': 'models/heroes/abaddon/abaddon.vmdl', 'model_scale': 0.6, 'texturename': 'abaddon', 'soundSet': 'Hero_Abaddon', 'movespeed': 325, 'healthBarOffset': -45}, 'alchemist': {'abilities': ['alchemist_acid_spray'], 'armor': 5, 'attackAnimationPoint': 0.35, 'attackRange': 1, 'attackRate': 1, 'damageMin': [50, 100, 200], 'damageMax': [70, 140, 280], 'displayName': '#dac_hero_name_alchemist', 'dota_unit_name': 'npc_dota_hero_alchemist', 'draftTier': 4, 'goldCost': 4, 'health': [2000, 4000, 8000], 'id': 46, 'hintW

First, we should create a class which makes it easy for us to access important information about each of the characters. 

In [0]:
class UnitInfo:
  def __init__(self):
    self.unit_info = unit_info

  # Return a list of synergies for a certain uit
  def get_synergies(self, unit):
    return self.unit_info[unit]['keywords'].split()

  # cost in gold of a unit
  def get_cost(self, unit):
    try:
      return self.unit_info[unit]['goldCost']
    except:
      return -1

In [8]:
info = UnitInfo()
info.get_synergies('dragon_knight')
info.get_cost('dragon_knight')

4

We should also create a game simulator class. For now, it will be really basic (so we can get a working prototype of the AI). 

TO DO: 


FIX ALLIANCES FOR NAGA, BRUTAL, ...

CHANGE OBS SPACE / STATE AND HOW MODEL PROCESSES IT (TIE WEIGHTS)

In [0]:
# This code cell is for creating a dictionary which assigns an alliance 
# to the number of different units needed for that alliance
# e.g Demon Hunter = 1, Knight = 2, Assassin = 3
from collections import defaultdict
import itertools

alliance_levels = {
    "assassin": {"boost":3,"max":6},
    "bloodbound": {"boost":2, "max":2},
    "brawny": {"boost":2, "max":4},
    "brute": {"boost":2,"max":4},
    "champion": {"boost":1, "max":1},
    "deadeye": {"boost":2, "max":2},
    "demon": {"boost":1, "max":1},
    "dragon": {"boost":2, "max":2},
    "druid": {"boost":2, "max":4},
    "dwarf": {"boost":3, "max":9},
    "ogre": {"boost":3,"max":9},
    "elf": {"boost": 3, "max": 9},
    "healer": {"boost":3, "max":3},
    "heartless": {"boost":2, "max":6},
    "human": {"boost":2, "max":6},
    "hunter": {"boost":3,"max":6},
    "insect": {"boost":2, "max":4},
    "inventor": {"boost":2, "max":4},
    "knight": {"boost":2, "max":6},
    "mage": {"boost":3,"max":6},
    "primordial": {"boost":2, "max":4},
    "savage": {"boost":2, "max":6},
    "naga": {"boost":2, "max":4}, 
    "goblin": {"boost":2, "max":6},
    "shaman": {"boost":2, "max":4}, 
    "spiritbrother" : {"boost" : 3, "max" : 9},
    "troll": {"boost":2, "max":4},
    "undead" : {"boost" : 3, "max" : 9}, # double check
    "warlock": {"boost":2, "max":6},
    "warrior": {"boost":3,"max":6}
}

In [0]:
import math

Make sure to add the current level of each synergy for each unit in the store

In [0]:
# probability of tiers given level
level_prob = {
        '1' : [1, 0, 0, 0, 0],
        '2' : [0.7, 0.3, 0, 0, 0],
        '3' : [0.6, 0.35, 0.05, 0, 0],
        '4' : [0.5, 0.35, 0.15, 0, 0], 
        '5' : [	0.4, 0.35,	0.25,	0,	0], 
        '6' : [0.35, 0.30, 0.3, 0.05], 
        '7' : [0.25, 0.3, 0.35, 0.1, 0], 
        '8' : [0.22, 0.27, 0.35, 0.15, 0.1], 
        '9' : [0.2, 0.25, 0.3, 0.2, 0.3], 
        '10' : [0.15, 0.21, 0.28, 0.3, 0.06]
    }

# Create a dictionary which maps the cost / draft tier to a list of heroes 
# which are members of that tier
# e.g 4 = ['Dragon Knight', 'Kunkka', ...]
tier_to_unit = defaultdict(list)
for unit in unit_info:
  cost = info.get_cost(unit)
  if cost != -1 and cost != 0:
    tier_to_unit[str(cost)].append(unit)

class SimpleSimulator():
  def __init__(self):
    self.inventory = defaultdict(int)
    self.player_level = 1
    self.gold = 5
    self.xp = 1

    self.best_inv_synergies = {}
    # contains the best combination of pieces which can go on the board 
    self.best_working_inv = []
  
  def reset(self):
    self.inventory = defaultdict(int)
    self.player_level = 1
    self.gold = 5
    self.xp = 1

    self.best_inv_synergies = {}
    self.best_working_inv = []

  # WORKS AND IS DONE
  def generate_units(self, player_level): # player_level affects probability
    
    # get probability distribution of tiers based on player level
    probabilities = level_prob[str(player_level)]
    units = []

    from numpy.random import choice
    # get units of 5 slots which show up in store
    for i in range(5):
      # choose draft tier of slot i
      draw = choice([1, 2, 3, 4, 5], 1, p=probabilities)
      # choose a hero at random from the pool (account for pool size)
      unit = choice(tier_to_unit[str(draw[0])])
      units.append(unit) # add to the 5 slots

    return units # list of units in the shop

  # Returns tuple consisting of level 1s, 2s, and 3 for a count
  def calcLevel(self,count):
    l1 = 0
    l2 = 0
    l3 = 0
    if(count/9>1):
      # Level 3's
      l3 = math.floor(count/9)
      count -= 9*(math.floor(count/9))
    if(count/3>1):
      # Level 2's
      l2 = math.floor(count/3)
      count -= 3*(math.floor(count/3))
    
    # Level 1's
    l1 = count
    return ([l1,l2,l3])

  # Given the amount of spaces on the board, this function will find all 
  # combinations of pieces, and will choose the one with the maximum reward
  def calculate_reward(self):
    fullInventory=[]
    unitLevels={} # maps unit to its level
    for unit in self.inventory:
      unitCount = self.inventory[unit]
      unitLevels[unit] = self.calcLevel(unitCount)

    for unit in unitLevels:
      for index, level in enumerate(unitLevels[unit]):
        # ex. tusk1, tusk1, tusk2 corresponds to a unit count of 5, with 
        # 2 level 1 tusks and 1 level 2 tusk
        # those are added to fullInventory
        fullInventory.extend([(unit+str(index+1))]*level)

    # defining parallel array to all combos
    allComboSynergyLevel = []
    max_score = 0

    # choose a combination of player_level pieces from the fullInventory
    for combo in itertools.combinations(fullInventory,self.player_level):
      comboSynergy = defaultdict(int)
      score = 0

      # Goes through each synergy of each unit and tallies the frequency
      for unit in combo:
        # Account for level in score
        if unit[:-1] == '2':
          score += 70
        elif unit[:-1] == '3':
          score += 140

        # get the unit's name from the string and get its synergies
        unit = unit[:-1]
        synergies = info.get_synergies(unit)
        # add synergies 
        for synergy in synergies:
          comboSynergy[synergy] +=1

      # Goes through the frequencies of each synergy and calculates level and adds it to parallel array
      # holds levels for each synergy
      temp ={}
      for synergy in comboSynergy:
        # some synergies don't exist, need to be fixed
        try:
          boost=alliance_levels[synergy]["boost"]
          cap = alliance_levels[synergy]["max"]
        except: 
          boost = 2
          cap = 2

        # Calculate synergy level, and set its value in temp (a dictionary) 
        # to the corresponding boost it receives
        synergyLevel = comboSynergy[synergy]/boost
        if(synergyLevel <1):
          temp[synergy]=0
        elif(synergyLevel > cap/boost):
          temp[synergy] = cap
        else:
          temp[synergy] = math.floor(synergyLevel)

        score += 50 * temp[synergy]
      
      if score > max_score:
        max_score = score
        self.best_inv_synergies = comboSynergy
        self.best_working_inv = combo
      # add temp to the list of all combinations
      #allComboSynergyLevel.append(temp)
    return max_score

  # Returns observation space of network
  # Observation space size: 2 + 2 * 5 + 3 * 5 = 27 
  def get_obs_space(self, units):
    total_syn_count = defaultdict(int)
    for piece in self.inventory:
      if self.inventory[piece] > 0:
        for synergy in info.get_synergies(piece):
          total_syn_count[synergy] += 1

    # Instead of making the observation space the entire set of units,
    # we can make it related to just the units that are in the store
    obs_space = []
    obs_space.append(self.gold)
    obs_space.append(self.player_level)
  
    # How do you determine how many synergies already exist for the particular unit?
    for unit in units:
      obs_space.append(info.get_cost(unit)) # cost of unit
      obs_space.append(self.inventory[unit]) # how many pieces already exist

      synergies = info.get_synergies(unit)
      # how many of the given synergies are there (but what if the other contributing pieces are useless?)
      for i, synergy in enumerate(synergies):
        # involve the # of pieces needed for a boost as a factor
        try:
          obs_space.append(total_syn_count[synergy] * alliance_levels[synergy]['boost'])
        except:
          obs_space.append(total_syn_count[synergy] * 3)
      if i <= 1: # zero pad since some units have 3 synergies
        obs_space.append(0)

      # the number of pieces already existing implicitly tells the model that
      # if there is 0 then it shouldn't count to the synergy
      '''
      # 1 if exists
      if self.inventory[unit] > 0:
        obs_space.append(1)
      else:
        obs_space.append(0)'''

    return obs_space

  # actions - what to do 
  def round(self, units, actions, rnd):
    reward_before = self.calculate_reward()
    current_state = self.get_obs_space(units)
    
    for action in actions:
      # if a unit is being bought, add it to the inventory
      if action[:3] == 'buy':
        self.inventory[action[3:]] += 1
      elif action[:4] == 'sell':
        # Here, we account for the fact that technically 3 units are being sold
        # if a level 2 unit is sold
        # However,  if there are 4 pieces, that means there is 1 level 2 and 1 
        # level 1, so we make the assumption that the level 1 is sold
        if self.inventory[action[4:]] % 3 == 0 and self.inventory[action[4:]] != 9:
          self.inventory[action[4:]] -= 3
          self.gold += (3 * info.get_cost(action[4:]))
        # if the unit is level 9, sell all of them
        elif self.inventory[action[4:]] == 9:
          self.inventory[action[4:]] -= 9
          self.gold += (9 * info.get_cost(action[4:]))
        else:
          self.inventory[action[4:]] -= 1
          self.gold += info.get_cost(action[4:])
      # invariant -> enough gold
      elif action == 'level':
        while self.xp <= (2 ** self.level):
          self.xp += 4
          self.gold -= 5
        self.player_level += 1
        # if 39/40 and level up, then the player is level 11 with 3 leftover xp
        self.xp = self.xp % (2 ** self.level)


    self.xp += 1
    if self.xp == 2 ** self.player_level:
      self.player_level += 1
      self.xp = 0 
    self.gold += (5 + self.gold * 0.1) 
    
    reward_after = self.calculate_reward()
    reward = reward_after - reward_before
    
    current_new_state = self.get_obs_space(units)

    return (current_state, actions, reward, current_new_state, rnd == 35)
  

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from collections import deque
import keras
import numpy as np

REPLAY_MEM_SIZE = 50000
MIN_REPLAY_MEM_SIZE = 50
MINIBATCH_SIZE = 25
DISCOUNT = 0.95
OBSERVATION_SPACE_SIZE = (27, )
ACTION_SPACE_SIZE = 5
UPDATE_TARGET_EVERY = 5

class DQNAgent:
  def __init__(self):
    # main model
    self.model = self.create_model()
  
    self.target_model = self.create_model()
    self.target_model.set_weights(self.model.get_weights())
  
    self.replay_memory = deque(maxlen=REPLAY_MEM_SIZE)

    # when to update the target network with the main model's weights
    self.target_update_counter = 0

  # (observation space, action, reward, new observation space, done)
  def update_replay_memory(self, transition):
    self.replay_memory.append(transition)

  def create_model(self):
    
    model = Sequential()
    model.add(keras.layers.Input(shape=(OBSERVATION_SPACE_SIZE)))
    model.add(keras.layers.Reshape((9, 3)))
    model.add(Conv2D(5, 1, strides=1, padding='same'))
    model.add(keras.layers.Flatten())

    model.add(Dense(300)) # , input_shape=OBSERVATION_SPACE_SIZE))
    
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Dense(ACTION_SPACE_SIZE, activation='linear'))  # ACTION_SPACE_SIZE = how many choices (9)
    model.compile(loss="mse", optimizer=Adam(lr=0.001), metrics=['accuracy'])
    return model
  
  # train main network every step per episode
  
  def train(self, terminal_state, step):
    import random

    if len(self.replay_memory) < MIN_REPLAY_MEM_SIZE:
      return
    
    minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)

    # gold, count of each alliance, etc.
    # Current qs list is the q value the model assigns to each action
    current_states = np.array([transition[0] for transition in minibatch])
    current_qs_list = self.model.predict(current_states)

    # Get future states from minibatch, query model for NN values
    # when using target model, query it -
    new_current_states = np.array([transition[3] for transition in minibatch])
    future_qs_list = self.target_model.predict(new_current_states)

    X = []
    y = []

    for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):
      # If not terminal state, get new q from future states; else set it to 0 
      if not done:
        # Using the action w/ the max future Q value, set the new q value using 
        # a discount
        max_future_q = np.max(future_qs_list[index])
        new_q = reward + DISCOUNT * max_future_q
      else:
        new_q = reward

      # Update q-value for given state
      current_qs = current_qs_list[index] # get example from minibatch
      # Map action (such as buy unit ...) to number / index
      try: 
        current_qs[action] = new_q # update
      except:
        print('The action is' , action)

      # Add the state
      X.append(current_state)
      # Using the updated / new list of rewards / q-values, add it to the 
      # label 
      y.append(current_qs)
    
    # fit on all samples as one batch
    self.model.fit(np.array(X), np.array(y), batch_size=MINIBATCH_SIZE, verbose=0, shuffle=False)
    
    if terminal_state:
      self.target_update_counter += 1

    # If counter reaches threshold, update target model w/ main model's weights
    if self.target_update_counter > UPDATE_TARGET_EVERY:
      self.target_model.set_weights(self.model.get_weights())
      self.target_update_counter = 0

  def get_qs(self, state):
    # must change
    if np.shape(state) == (27, ):
      return self.model.predict(np.array([state]))[0]
    return self.model.predict(np.array(state))[0]

In [17]:
agent = DQNAgent()

TypeError: ignored

In [0]:
from tqdm import tqdm

EPISODES = 100
epsilon = 0.99
total_episode_reward = 0
# Iterate over episodes
for episode in tqdm(range(1, EPISODES + 1), ascii=True, unit='episodes'):
  episode_reward = 0
  rnd = 1

  env = SimpleSimulator()
  env.reset()
  state = [0, 0, 0, 0, 0]

  for rnd in range(1, 36):
    units = env.generate_units(env.player_level)

    if np.random.random() > epsilon:
      action = np.argmax(agent.get_qs(current_state))
    else:
      action = np.random.randint(0, ACTION_SPACE_SIZE)
    
    # Determine what the numerical action means (translate)
    # Since multiple actions can be taken in one step, make it a list
    # Let 0 be no action (zero pad)
    actions = ['buy' + units[action]]

    # empty param should be action, which we do not change since we want 
    # to represent it numerically instead of like a string
    current_state, _ , reward, new_state, done = env.round(units, actions, rnd)
    episode_reward += reward

    agent.update_replay_memory((current_state, action, reward, new_state, done))
    agent.train(done, round)

    current_state = new_state

    total_episode_reward += episode_reward


In [0]:
episode_reward
total_episode_reward

In [0]:
#np.argmax(agent.get_qs([1, 2, 3, 4, 5]))